In [73]:
import numpy as np
import pandas as pd
import os
import re
import csv
from datetime import datetime
from gensim.models import word2vec
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.layers import Input, Dense, Dropout, Flatten, Activation
from keras.callbacks import EarlyStopping
from keras.models import load_model
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split

In [74]:
pd.set_option('display.max_colwidth', 500)

In [75]:
smile_data=pd.read_csv('smile_data.csv',index_col=False)

In [76]:
smile_data.head()

,smiles,target1,target2,target3,target4,target5,target6,target7,target8,target9,target10,target11,target12
0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,CCN1C(=O)NC(c2ccccc2)C1=O,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
2,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]3CC[C@@]21C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
3,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
4,CC(O)(P(=O)(O)O)P(=O)(O)O,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0


In [77]:

smile_data_copy=smile_data[["smiles","target1"]]

mol=smile_data['smiles']


In [78]:
#smile_data.head()

In [79]:
smile_data_copy['smiles_lst']=smile_data_copy['smiles'].apply(lambda x:' '.join(list(x)))

E:\program_files\Anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
smile_data_copy.head()

,smiles,target1,smiles_lst
0,CCOc1ccc2nc(S(N)(=O)=O)sc2c1,0.0,C C O c 1 c c c 2 n c ( S ( N ) ( = O ) = O ) s c 2 c 1
1,CCN1C(=O)NC(c2ccccc2)C1=O,0.0,C C N 1 C ( = O ) N C ( c 2 c c c c c 2 ) C 1 = O
2,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]3CC[C@@]21C,NaN,C C [ C @ ] 1 ( O ) C C [ C @ H ] 2 [ C @ @ H ] 3 C C C 4 = C C C C [ C @ @ H ] 4 [ C @ H ] 3 C C [ C @ @ ] 2 1 C
3,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C,0.0,C C C N ( C C ) C ( C C ) C ( = O ) N c 1 c ( C ) c c c c 1 C
4,CC(O)(P(=O)(O)O)P(=O)(O)O,0.0,C C ( O ) ( P ( = O ) ( O ) O ) P ( = O ) ( O ) O


In [81]:
smile_data_copy_1col=pd.DataFrame(data=smile_data_copy,columns=['smiles_lst'])

In [82]:
smile_data_copy_1col.to_csv("whole_sens",index=None,header=False,sep=" ",quoting=csv.QUOTE_NONE,escapechar=' ')

In [83]:
sent=word2vec.Text8Corpus("whole_sens")

In [84]:
model=word2vec.Word2Vec(sent,seed=12345,min_count=1,size=300,sg=1,hs=1,negative=0,window=3)

In [85]:
model

In [86]:
model.wv.save_word2vec_format('vect2.txt',binary=False)

In [87]:
text_in=pd.read_csv("vect2.txt",sep=" ",header=None,skiprows=1)

In [88]:
text_in.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Columns: 301 entries, 0 to 300
dtypes: float64(300), object(1)
memory usage: 129.5+ KB


In [89]:
text_in.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,C,0.035231,-0.003664,0.006568,0.089244,0.032026,0.002260,0.039171,-0.026706,-0.015678,...,0.054323,0.055149,-0.042487,-0.110403,-0.031201,0.011980,-0.005483,-0.004577,0.044318,-0.083485
1,c,-0.006778,0.002516,-0.024652,0.032363,0.041799,-0.037853,-0.205738,-0.048009,0.103387,...,0.000524,-0.011866,-0.024554,-0.127726,0.070179,0.108143,0.029766,0.001316,0.003443,-0.058763
2,(,0.057599,-0.023383,-0.007160,0.020140,0.040947,0.003109,-0.013595,-0.044177,0.043422,...,-0.008369,-0.047327,-0.054990,-0.085025,0.060349,0.086681,-0.037217,-0.007317,0.131630,0.021391
3,),0.018408,-0.008802,0.026052,0.077548,0.071187,-0.017335,-0.026423,-0.015111,0.002228,...,0.032837,-0.117190,-0.217289,-0.136737,0.024853,0.150488,-0.099715,0.012349,0.246787,0.218485
4,O,0.077943,0.028043,-0.004486,0.037855,0.012791,-0.036976,0.020910,0.031513,0.057218,...,-0.023723,-0.069720,-0.037398,-0.061026,0.052188,0.113649,-0.052007,0.027025,0.120493,0.145080


In [90]:
text_in_t=text_in.transpose()

In [91]:
text_in_t.head()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,C,c,(,),O,1,=,@,[,],...,9,T,K,G,t,L,V,Y,D,y
1,0.0352315,-0.0067775,0.0575986,0.0184076,0.0779429,0.0752808,-0.0144067,0.0518906,0.0618442,-0.0411165,...,0.115458,0.0448289,0.0545196,0.0737586,0.0535397,0.0680412,0.00412863,0.0300346,0.022105,0.0141082
2,-0.00366432,0.0025156,-0.0233826,-0.00880205,0.0280429,0.0356728,-0.036072,-0.0257599,0.0508241,0.0942187,...,0.0136258,0.0197058,0.042713,0.0364185,-0.00391857,0.0358148,-0.00271328,0.000169496,8.36317e-05,-0.00346933
3,0.00656818,-0.0246518,-0.0071602,0.0260521,-0.00448627,-0.0140849,-0.0146001,0.116205,-0.000723265,0.0520795,...,-0.0187532,-0.0350793,-0.0340524,-0.0418236,-0.0224566,-0.0406223,-0.00414682,-0.0129198,-0.0108417,-0.00896201
4,0.0892436,0.0323625,0.0201399,0.0775482,0.037855,0.113825,0.137379,-0.193967,-0.0276527,0.0114485,...,0.00139642,0.132143,0.095686,0.116432,0.0818904,0.130782,0.0104796,0.0437235,0.0322006,0.0278638


In [92]:
text_in_t.columns = text_in_t.iloc[0]

In [93]:
text_in_t.reindex(text_in_t.index.drop(0)).head()

,C,c,(,),O,1,=,@,[,],...,9,T,K,G,t,L,V,Y,D,y
1,0.0352315,-0.0067775,0.0575986,0.0184076,0.0779429,0.0752808,-0.0144067,0.0518906,0.0618442,-0.0411165,...,0.115458,0.0448289,0.0545196,0.0737586,0.0535397,0.0680412,0.00412863,0.0300346,0.022105,0.0141082
2,-0.00366432,0.0025156,-0.0233826,-0.00880205,0.0280429,0.0356728,-0.036072,-0.0257599,0.0508241,0.0942187,...,0.0136258,0.0197058,0.042713,0.0364185,-0.00391857,0.0358148,-0.00271328,0.000169496,8.36317e-05,-0.00346933
3,0.00656818,-0.0246518,-0.0071602,0.0260521,-0.00448627,-0.0140849,-0.0146001,0.116205,-0.000723265,0.0520795,...,-0.0187532,-0.0350793,-0.0340524,-0.0418236,-0.0224566,-0.0406223,-0.00414682,-0.0129198,-0.0108417,-0.00896201
4,0.0892436,0.0323625,0.0201399,0.0775482,0.037855,0.113825,0.137379,-0.193967,-0.0276527,0.0114485,...,0.00139642,0.132143,0.095686,0.116432,0.0818904,0.130782,0.0104796,0.0437235,0.0322006,0.0278638
5,0.0320261,0.0417988,0.0409467,0.0711868,0.0127912,0.0492778,0.0191123,-0.190999,-0.0463808,0.0339399,...,-0.0257521,0.0395714,0.0370023,0.0323154,0.0142512,0.0412599,0.00338381,0.0118114,0.0088853,0.0116083


In [94]:
text_in_t_clmn=text_in_t.reindex(text_in_t.index.drop(0))

In [95]:
text_in_t_clmn_dict=text_in_t_clmn.to_dict(orient='list')

In [96]:


def add_lst(chek_lst):
    zarray = np.zeros(300)
    for elment in str(chek_lst).split(" "):
        zarray+=text_in_t_clmn_dict[elment]
    return np.array(zarray)

In [97]:
smile_data_copy['smiles_lst_vect_sum']=smile_data_copy['smiles_lst'].apply(add_lst)

E:\program_files\Anaconda3\envs\my-rdkit-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [98]:
df1=pd.DataFrame(smile_data_copy['smiles_lst_vect_sum'].values.tolist())

In [99]:
df1.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.838974,0.136342,-0.322644,1.999621,0.844633,-0.770592,-2.297244,-0.748412,1.825505,0.822230,...,0.539456,-1.179721,-1.640213,-3.433305,1.340281,3.699875,-0.339940,0.342347,2.166096,1.397608
1,0.722176,0.222568,-0.229151,1.640818,0.795344,-0.354779,-1.660152,-0.637040,1.222435,0.573799,...,0.634202,-0.513186,-1.536849,-3.087613,0.926914,2.667789,-0.197502,0.127673,1.480351,0.155448
2,2.460462,0.599181,1.263995,0.323851,-2.114013,-0.156931,0.282253,-1.615816,-0.664589,-0.031444,...,3.559223,0.098060,-3.643191,-6.470021,-0.393228,4.905343,-2.155158,-0.217973,7.147344,3.733098
3,0.910854,0.061927,-0.116086,2.194639,1.299302,-0.263369,-1.233103,-0.875932,0.954426,0.426825,...,0.850284,-0.257415,-2.143095,-3.677138,0.667448,2.506144,-0.431095,-0.005734,2.137851,-0.335053
4,0.707547,-0.214093,0.010068,1.819672,1.007854,-0.571129,-0.005389,-0.235526,0.669733,0.461226,...,0.248420,-1.761905,-2.016998,-2.386651,1.021458,2.704537,-1.374886,0.471308,3.467430,2.272898


In [100]:
smile_data_copy_label = smile_data_copy.drop(smile_data_copy.columns[[0,2,3]], axis=1) 

In [101]:
smile_data_copy_label_feat_combine=pd.concat([smile_data_copy_label,df1],axis=1)

In [102]:
smile_data_copy_label_feat_combine_nomiss=smile_data_copy_label_feat_combine.dropna(how='any')

In [103]:
df=smile_data_copy_label_feat_combine_nomiss

In [104]:
df.columns = df.columns.astype(str)

In [105]:
df.target1.value_counts()

0.0    7129
1.0     310
Name: target1, dtype: int64

In [106]:

def build_dnn(layers_num):#特征个数，128*128
    model_bp_ks = keras.Sequential()
    model_bp_ks.add(Dense(layers_num[1], input_shape=(layers_num[0],)))
    # model_bp_ks.add(regularizers.l1(0.0001))
    model_bp_ks.add(Activation('relu'))
    model_bp_ks.add(Dropout(0.5))

    model_bp_ks.add(Dense(layers_num[2]))
    # model_bp_ks.add(regularizers.l1(0.001))
    model_bp_ks.add(Activation('relu'))
    model_bp_ks.add(Dropout(0.5))

    # model_bp_ks.add(Dense(1))
    # model_bp_ks.add(Activation('linear'))

    model_bp_ks.add(Dense(1))
    model_bp_ks.add(Activation('sigmoid'))
    model_bp_ks.summary()  # print每层的参数
    # model_bp_ks.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    # model_bp_ks.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    # sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    # model_bp_ks.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model_bp_ks.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

    return model_bp_ks
df = df.dropna(how = "any",axis = 0)
feat = [x for x in df.columns.tolist() if x not in ("target1")]
x_train = df[feat]
y_train = df["target1"]
x_train_used, x_validation, y_train_used, y_validation = train_test_split(x_train,
                                                                                  y_train,
                                                                                  test_size=0.4,
                                                                                  random_state=1)

estimators = build_dnn([x_train_used.shape[1], 128, 128])
# estimators = build_dnn([x_train_used.shape[1], 128, 128])
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

estimators.fit(x_train_used,
               y_train_used,
               batch_size=32,
               epochs=100,
               verbose=2,
               # shuffle=False,
               validation_data=(x_validation, y_validation),
               callbacks=[early_stopping]
               # validation_split=0.05
               )
               
y_pred_train_all_prob = estimators.predict_proba(x_train_used)
best_threshold = 0.3
y_pred_label_train = np.where(y_pred_train_all_prob > best_threshold, 1, 0)

def evaluation(y_real, y_pred):
    accuracy_ = accuracy_score(y_real, y_pred)
    precision_ = precision_score(y_real, y_pred)
    recall_score_ = recall_score(y_real, y_pred)
    f1_score_ = f1_score(y_real, y_pred)
    # auc_ = metrics.roc_curve(y_test.values, ypred_label, pos_label=1)
    fpr, tpr, thresholds = metrics.roc_curve(y_real, y_pred, pos_label=1)
    auc_ = metrics.auc(fpr, tpr)
    return accuracy_, precision_, recall_score_, auc_, f1_score_

accuracy_t, precision_t, recall_score_t, auc_, f1_score_t = evaluation(y_train_used, y_pred_label_train)
performance_train=pd.DataFrame()
performance_train.loc[0, "accuracy"] = accuracy_t
performance_train.loc[0, "precision"] = precision_t
performance_train.loc[0, "recall_score"] = recall_score_t
# performance_train_df.loc[i, "auc"] = auc_t
performance_train.loc[0, "f1_score"] = f1_score_t

print("accuracy:{} precision:{} recall_score:{} f1_score:{} "
      .format(accuracy_t, precision_t, recall_score_t, f1_score_t))

y_pred_train_all_prob = estimators.predict_proba(x_validation)
best_threshold = 0.5
y_pred_label_train = np.where(y_pred_train_all_prob > best_threshold, 1, 0)
      
accuracy_t, precision_t, recall_score_t, auc_, f1_score_t = evaluation(y_validation, y_pred_label_train)
performance_train.loc[1, "accuracy"] = accuracy_t
performance_train.loc[1, "precision"] = precision_t
performance_train.loc[1, "recall_score"] = recall_score_t
# performance_train_df.loc[i, "auc"] = auc_t
performance_train.loc[1, "f1_score"] = f1_score_t

print("accuracy:{} precision:{} recall_score:{} f1_score:{} "
      .format(accuracy_t, precision_t, recall_score_t, f1_score_t))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               38400     
_________________________________________________________________
activation_4 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_5 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
__________